# Getting Started in TINerator

Thanks for trying out TINerator. This notebook provides a soft introduction to the module, and is intended to demonstrate the workflow at a high level. More advanced capabilities can be found in the documentation.

## Workflow Overview

The workflow will follow this pattern:

0. Import the TINerator module
1. Prepare the DEM that you wish to mesh
    - Read the DEM from disk
    - [optional] Clip the DEM using a watershed boundary shapefile
    - [optional] Reproject the DEM from a lat-lon into a meter-based CRS
    - [optional] Post-process the DEM using a pit-filling algorithm
2. Prepare the feature that you wish to refine the mesh around
    - This can be a flowline shapefile, or
    - Watershed delineation can be performed to extract the flowline from the DEM
3. Generate a triangulation from the DEM
4. Extrude the triangulation into the subsurface with layering
5. [optional] Add material attributes to the extruded mesh
6. [optional] Define face sets / point sets / element sets to act as "regions"
7. Write the mesh to disk

## 0. Import the TINerator module

In [1]:
import tinerator as tin
from tinerator import examples

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


## 1. Prepare the DEM that you wish to mesh

TINerator comes with some built-in datasets. We'll use the New Mexico dataset for this example.
It contains the following paths to files:

| class attribute | description |
| --------------- | ----------- |
| `dem` | path to a DEM tile |
| `watershed_boundary` | path to the watershed boundary shapefile |
| `flowline` | path to the flowline shapefile |

First, load the DEM from disk:

In [2]:
dem = tin.gis.load_raster(examples.new_mexico.dem)
print(dem)
print(f'\nNoData value: {dem.no_data_value}\n')
print(f'CRS: {dem.crs}')

Raster<data.shape=(2373, 2575), extent=[-107.70482, 36.29657, -107.46639, 36.5163], CRS="NAD83">

NoData value: -3.4028230607370965e+38

CRS: GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4269"]]


Next, load the watershed boundary and clip the DEM with it:

In [4]:
dem.plot()

In [6]:
ws_bnd = tin.gis.load_shapefile(examples.new_mexico.boundary)
print(ws_bnd)
print(f"\nCRS: {ws_bnd.crs}\n")
print(f"Shapefile properties: {ws_bnd.properties}\n")

# We can add the `layers` keyword to plot to add
# multiple objects to plot at once
ws_bnd.plot(layers=[dem])

Geometry<"Polygon", shapes=1, crs="NAD83", properties=1>

CRS: GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4269"]]

Shapefile properties: {'properties': {'id': [None]}, 'metadata': {'schema': OrderedDict([('id', 'int:10')])}}



Next, let's reproject the raster into meters. Meters are the units that Amanzi-ATS expects, and moving out of lat-lon helps to avoid floating-point overflow errors when triangulating.

In [8]:
# Clip the DEM!
dem = tin.gis.clip_raster(dem, ws_bnd)
dem.plot(layers=[ws_bnd])

In [9]:
print(f"Units before reprojecting: {dem.units}")
dem = tin.gis.reproject_raster(dem, "epsg:32112")
print(f"Units after reprojecting: {dem.units}")

Units before reprojecting: degree
Units after reprojecting: metre


Finally, use `fill_depressions` to fill pits and flats:

In [10]:
# Note that fill_depressions operates in-place: it doesn't return a new Raster object
dem.fill_depressions()

# Optionally, we can also save the post-processed DEM:
# dem.save("dem_clipped_reprojected_pit-filled.tif")


A Priority-Flood (Zhou2016 version)
C Zhou, G., Sun, Z., Fu, S., 2016. An efficient variant of the Priority-Flood algorithm for filling depressions in raster digital elevation models. Computers & Geosciences 90, Part A, 87 – 96. doi:http://dx.doi.org/10.1016/j.cageo.2016.02.021

t Zhou2016 wall-time = 0.201451 s

A Barnes (2014) Flat Resolution Flat Mask Generation
C Barnes, R., Lehman, C., Mulla, D., 2014a. An efficient assignment of drainage direction over flat surfaces in raster digital elevation models. Computers & Geosciences 62, 128–135. doi:10.1016/j.cageo.2013.01.009

t Succeeded in = 0.0115825 s
p Setting up labels matrix...
p Setting up flat resolution mask...
p Searching for flats...
t Succeeded in = 0.0191157 s================= ] (99% - 0.0s - 1 threads)
m Cells with no flow direction = 19050
m Low edge cells               = 5708
m High edge cells              = 16532
p Labeling flats...
m Unique flats = 5313
p Removing flats without outlets from the queue...
  The flat he

### 2. Prepare the feature that you wish to refine the mesh around

When creating a triangulation, a distance field is created, taking a copy of the raster and filling
every cell with the Euclidean distance to the refinement feature. The distance field then informs how the triangulation algorithm should refine triangles.

The refinement feature is a Geometry object loaded from a shapefile or created by the user. Consequently, the refinement feature can be a collection of points, lines, or polygons.

In the case of flowlines, they will be lines. We can either read these from a shapefile on disk, or create them using the built-in watershed delineation method.

In [12]:
flowlines_shapefile = tin.gis.load_shapefile(examples.new_mexico.flowline)
flowlines_shapefile.plot(layers=[dem])

### 2.1 Watershed delineation

Watershed delineation is performed using the `tinerator.gis.watershed_delineation` function.

There are a few optional parameters:

- `method`: by default, uses the D8 algorithm. Also supports D4, Quinn, Rho8, Rho4, and more.
- `threshold`: a flow accumulation raster is generated from running watershed delineation algorithm. As flow is simulated to the cells in the raster, each cell will have a value containing the amount of flow into it. `threshold` "cuts off" cells with a value below this and keeps cells with a value above it, where all kept cells form the flowline.

In [13]:
flowlines_wsdelin = tin.gis.watershed_delineation(dem)
# flowlines_wsdelin = tin.gis.watershed_delineation(dem, threshold=15000., method='D4')
flowlines_wsdelin.plot(layers=[dem])


A O'Callaghan (1984)/Marks (1984) D8/D4 Flow Accumulation
C O'Callaghan, J.F., Mark, D.M., 1984. The Extraction of Drainage Networks from Digital Elevation Data. Computer vision, graphics, and image processing 28, 323--344.

c topology = D8
[================================================= ] (99% - 0.0s - 1 threads)
A Generic Flow Accumulation Algorithm
p Creating dependencies array...

d Source cells found = 271766
p Calculating flow accumulation...
t Wall-time       = 0.122951 s


[16:33:01] Threshold was not set by user; set automatically to:        ]8;id=1631831581.364533-755464;file:///Users/livingston/dev/lanl/tinerator/tinerator/gis/watershed_delin.py\watershed_delin.py]8;;\:197
           20276.52695. Adjust this value to adjust the river network.

### 3. Generate a triangulation from the DEM

The core of the triangulation algorithm are the parameters:

- `min_edge_length`: The minimum edge length for created triangles.
- `max_edge_length`: The maximum edge length for created triangles.
- `scaling_type`: Can be "relative" or "absolute". If absolute, then the edge length parameters will interpreted as meters (or whatever the units of the DEM are). If "relative", then the edge length parameters will be considered *percent of the DEM extent*.

Recall that the refinement feature (the flowlines above) create a distance field internally. That distance field is normalized to `[0., 1.]`.

Where the distance field is `0.`, triangles will have an edge length of `min_edge_length`. Where the distance field is `1.`, triangles will have an edge length of `max_edge_length`. Distance field values between `[0., 1.]` will linearly interpolate the edge lengths.

Various triangulation kernels are supported:

- JIGSAW
- MeshPy (based on Triangle)
- Gmsh

In [15]:
mesh_tri = tin.meshing.triangulate(
    dem,
    min_edge_length = 0.005,
    max_edge_length = 0.1,
    scaling_type = "relative",
    refinement_feature = flowlines_wsdelin,
    method = "jigsaw",
)

mesh_tri.plot()

### 4. Extrude the triangulation into the subsurface with layering

### 5. [optional] Add material attributes to the extruded mesh

### 6. [optional] Define face sets / point sets / element sets to act as "regions"

### 7. Write the mesh to disk

In [9]:
#gis_triang = tin.gis.vectorize_triangulation(mesh_tri)
#gis_triang.save('triangulation.shp')

# ("layer type", layer data, sublayers, material_id)

layers = [
    ("constant", 500., 4, 1),
    ("constant", 200., 1, 2),
    ("snapped", 1000., 3, [3,2,1])
]

volume_mesh = tin.meshing.extrude_mesh(
    mesh_tri, 
    layers
)

volume_mesh.view(window_size=(800,800))

# TODO: add attributes
# TODO: add facesets

volume_mesh.save("vol_mesh.vtk")
volume_mesh.save("vol_mesh.exo")
volume_mesh.save("vol_mesh.inp")


You are using exodus.py v 1.20.2 (seacas-py3), a python wrapper of some of the exodus library.

Copyright (c) 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021 National Technology &
Engineering Solutions of Sandia, LLC (NTESS).  Under the terms of
Contract DE-NA0003525 with NTESS, the U.S. Government retains certain
rights in this software.

Opening exodus file: vol_mesh.exo
Closing exodus file: vol_mesh.exo
EXODUS write was successful.
